# KDD 2020 
# Where should I park?
Using RAPIDS to find parking spots in Seattle.

## Load the modules

In [1]:
!nvidia-smi

Sat Aug 22 02:15:52 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.06    Driver Version: 450.51.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  TITAN RTX           Off  | 00000000:01:00.0 Off |                  N/A |
| 41%   33C    P8    24W / 280W |      0MiB / 24217MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Import modules

In [3]:
import cudf
from collections import OrderedDict
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt

%load_ext autotime

In [4]:
print(cudf.__version__)

0.15.0a+4954.ga5dda7faf
time: 443 µs


# Download the data
If necessary, download the data from my website and unpack. Note -- this may take around 10 minutes depending on the speed of your Internet connection.

In [5]:
import os

directory = os.path.exists('../data')
archive   = os.path.exists('../data/parking_MayJun2019.tar.gz')
file      = os.path.exists('../data/parking_MayJun2019.csv')

time: 4.09 ms


In [6]:
if not directory:
    os.mkdir('../data')

if not archive and not file:
    import wget, shutil
    
    def bar_custom(current, total, width=80):
            print('Downloading: %d%% [%d / %d] bytes' % (current / total * 100.0, current, total))
        
    wget.download('http://tomdrabas.com/data/seattle_parking/parking_MayJun2019.tar.gz')
    shutil.move('parking_MayJun2019.tar.gz', '../data/parking_MayJun2019.tar.gz')
    
if not file:
    import tarfile

    tf = tarfile.open('../data/parking_MayJun2019.tar.gz')
    tf.extractall(path='../data/')

time: 3.63 ms


# Read the data

In [7]:
!head -n 10 ../data/parking_MayJun2019.csv

OccupancyDateTime,PaidOccupancy,BlockfaceName,SideOfStreet,SourceElementKey,ParkingTimeLimitCategory,ParkingSpaceCount,PaidParkingArea,PaidParkingSubArea,PaidParkingRate,ParkingCategory,Location,dow
2019-05-24T08:35:00,1,SENECA ST BETWEEN BOYLSTON AVE AND HARVARD AVE,SE,35438,120,4,First Hill,,,Paid Parking,POINT (-122.32245377 47.61236006),4
2019-05-24T19:57:00,3,REPUBLICAN ST BETWEEN QUEEN ANNE AVE N AND 1ST AVE N,S,57862,120,3,Uptown,Core,,Paid Parking,POINT (-122.35604157 47.62320827),4
2019-05-24T11:21:00,1,8TH AVE BETWEEN PINE ST AND OLIVE WAY,NE,53542,120,4,Commercial Core,Retail,,Paid Parking,POINT (-122.33353011 47.61360677),4
2019-05-24T11:56:00,2,BLANCHARD ST BETWEEN 1ST AVE AND 2ND AVE,SE,9354,120,4,Belltown,South,,Paid Parking,POINT (-122.3446913 47.61292255),4
2019-05-24T10:57:00,13,VALLEY ST BETWEEN MINOR AVE N AND YALE AVE N,N,35889,600,7,South Lake Union,North,,Paid Parking,POINT (-122.33168043 47.62602813),4
2019-05-24T09:23:00,1,PIKE ST BETWEEN 4TH AVE AND 5TH AVE,NW

In [8]:
dtypes = OrderedDict([
    ('OccupancyDateTime', 'date'),
    ('PaidOccupancy', 'int64'),
    ('BlockfaceName', 'str'),
    ('SideOfStreet', 'str'),
    ('SourceElementKey', 'int64'),
    ('ParkingTimeLimitCategory', 'int64'),
    ('ParkingSpaceCount', 'int64'),
    ('PaidParkingArea', 'str'),
    ('PaidParkingSubArea', 'str'),
    ('PaidParkingRate', 'int8'),
    ('ParkingCategory', 'str'),
    ('Location', 'str'),
    ('dow', 'int8')
])

df = cudf.read_csv(
    '../data/parking_MayJun2019.csv'
    , skiprows=1
    , dtype=list(dtypes.values())
    , names=list(dtypes.keys())
)

df = df.fillna({'PaidOccupancy': 0, 'ParkingSpaceCount': 999, 'PaidParkingSubArea': 'UKN'})

time: 4.25 s


In [9]:
# size of the file
import os
print('Filesize: {0:.2f}GB'.format(os.path.getsize('../data/parking_MayJun2019.csv') / (1024 ** 3)))

Filesize: 6.78GB
time: 650 µs


In [10]:
df['PaidOccupancy']     = df['PaidOccupancy'].astype('float64')
df['ParkingSpaceCount'] = df['ParkingSpaceCount'].astype('float64')

time: 15.5 ms


In [11]:
df.dtypes

OccupancyDateTime           datetime64[ms]
PaidOccupancy                      float64
BlockfaceName                       object
SideOfStreet                        object
SourceElementKey                     int64
ParkingTimeLimitCategory             int64
ParkingSpaceCount                  float64
PaidParkingArea                     object
PaidParkingSubArea                  object
PaidParkingRate                       int8
ParkingCategory                     object
Location                            object
dow                                   int8
dtype: object

time: 4.21 ms


In [12]:
print('The dataset has {0:,} records and {1} columns.'.format(*df.shape))

The dataset has 48,675,039 records and 13 columns.
time: 4.5 ms


In [13]:
df.head().to_pandas()

,OccupancyDateTime,PaidOccupancy,BlockfaceName,SideOfStreet,SourceElementKey,ParkingTimeLimitCategory,ParkingSpaceCount,PaidParkingArea,PaidParkingSubArea,PaidParkingRate,ParkingCategory,Location,dow
0,2019-05-24 08:35:00,1.0,SENECA ST BETWEEN BOYLSTON AVE AND HARVARD AVE,SE,35438,120,4.0,First Hill,UKN,NaN,Paid Parking,POINT (-122.32245377 47.61236006),4
1,2019-05-24 19:57:00,3.0,REPUBLICAN ST BETWEEN QUEEN ANNE AVE N AND 1ST...,S,57862,120,3.0,Uptown,Core,NaN,Paid Parking,POINT (-122.35604157 47.62320827),4
2,2019-05-24 11:21:00,1.0,8TH AVE BETWEEN PINE ST AND OLIVE WAY,NE,53542,120,4.0,Commercial Core,Retail,NaN,Paid Parking,POINT (-122.33353011 47.61360677),4
3,2019-05-24 11:56:00,2.0,BLANCHARD ST BETWEEN 1ST AVE AND 2ND AVE,SE,9354,120,4.0,Belltown,South,NaN,Paid Parking,POINT (-122.3446913 47.61292255),4
4,2019-05-24 10:57:00,13.0,VALLEY ST BETWEEN MINOR AVE N AND YALE AVE N,N,35889,600,7.0,South Lake Union,North,NaN,Paid Parking,POINT (-122.33168043 47.62602813),4


time: 35.3 ms


# Extract date information

In [14]:
df['year'] = df['OccupancyDateTime']._column.year
df['month'] = df['OccupancyDateTime']._column.month
df['day'] = df['OccupancyDateTime']._column.day

df['hour'] = df['OccupancyDateTime']._column.hour
df['minute'] = df['OccupancyDateTime']._column.minute

df[['OccupancyDateTime','year','month','day','hour', 'minute']].head().to_pandas()

,OccupancyDateTime,year,month,day,hour,minute
0,2019-05-24 08:35:00,2019,5,24,8,35
1,2019-05-24 19:57:00,2019,5,24,19,57
2,2019-05-24 11:21:00,2019,5,24,11,21
3,2019-05-24 11:56:00,2019,5,24,11,56
4,2019-05-24 10:57:00,2019,5,24,10,57


time: 19.5 ms


In [15]:
counts = df.groupby(['year', 'month', 'day']).agg({'OccupancyDateTime': 'count'})
counts

OccupancyDateTime
year month day                   
2019 5     1               959957
           2               960058
           3               959713
           4               970780
           6               953855
           7               953749
           8               959130
           9               954661
           10              959962
           11              972658
           13              949417
           14              958798
           15              956968
           16              958204
           17              960651
           18              957570
           20              956194
           21              962393
           22              962377
           23              956228
           24              954530
           25              963938
           28              950981
           29              954106
           30              959746
           31              959143
     6     1               966546
           3               946295
           4               954473
           5               957349
           6               956514
           7               958960
           8               970800
           10              952197
           11              946510
           12              952578
           13              951078
           14              948080
           15              958350
           17              942704
           18              951444
           19              948173
           20              948713
           21              941868
           22              953226
           24              938273
           25              935960
           26              941543
           27              944252
           28              939084
           29              944302

time: 66.4 ms


In [16]:
print('Average number of transactions per day: {0:,.0f}'.format(counts['OccupancyDateTime'].mean()))

Average number of transactions per day: 954,413
time: 814 µs


# All parking locations

In [17]:
locations = df[['SourceElementKey', 'BlockfaceName', 'SideOfStreet',
       'ParkingTimeLimitCategory', 'ParkingSpaceCount',
       'PaidParkingArea', 'PaidParkingSubArea',
       'ParkingCategory', 'Location']].drop_duplicates()

locations.head().to_pandas()

,SourceElementKey,BlockfaceName,SideOfStreet,ParkingTimeLimitCategory,ParkingSpaceCount,PaidParkingArea,PaidParkingSubArea,ParkingCategory,Location
4080,1001,1ST AVE BETWEEN CHERRY ST AND COLUMBIA ST,SW,120,4.0,Pioneer Square,Core,Paid Parking,POINT (-122.33469356 47.6028728)
1336,1002,1ST AVE BETWEEN CHERRY ST AND COLUMBIA ST,NE,120,8.0,Pioneer Square,Core,Paid Parking,POINT (-122.33451266 47.60294861)
4455,1006,1ST AVE BETWEEN COLUMBIA ST AND MARION ST,NE,120,7.0,Commercial Core,Waterfront,Paid Parking,POINT (-122.33514326 47.60367439)
1026,1009,1ST AVE BETWEEN MADISON ST AND SPRING ST,SW,30,5.0,Commercial Core,Waterfront,Paid Parking,POINT (-122.3366575 47.60501765)
3208,1010,1ST AVE BETWEEN MADISON ST AND SPRING ST,NE,120,5.0,Commercial Core,Waterfront,Paid Parking,POINT (-122.33644748 47.6051007)


time: 6.61 s


In [18]:
print('Number of parking locations in Seattle: {0:,}'.format(locations.shape[0]))

Number of parking locations in Seattle: 1,528
time: 516 µs


In [19]:
def extractLon(location):
    lon = location.str.extract('([0-9\.\-]+) ([0-9\.]+)')[0]
    return lon#.stod()

def extractLat(location):
    lon = location.str.extract('([0-9\.\-]+) ([0-9\.]+)')[1]
    return lon#.str.stod()

    
locations['longitude'] = extractLon(locations['Location']).astype('float')
locations['latitude'] = extractLat(locations['Location']).astype('float')

locations[['Location', 'longitude', 'latitude']].head().to_pandas()

,Location,longitude,latitude
4080,POINT (-122.33469356 47.6028728),-122.334694,47.602873
1336,POINT (-122.33451266 47.60294861),-122.334513,47.602949
4455,POINT (-122.33514326 47.60367439),-122.335143,47.603674
1026,POINT (-122.3366575 47.60501765),-122.336658,47.605018
3208,POINT (-122.33644748 47.6051007),-122.336447,47.605101


time: 19 ms


# Average occupancy

In [20]:
def avgOccupancy(PaidOccupancy, ParkingSpaceCount, AvgOccupancy):
    for i, (paid, available) in enumerate(zip(PaidOccupancy, ParkingSpaceCount)):
        AvgOccupancy[i] = min(1.0, paid / available) # cap it at 100%, sometimes we see more paid occupancy than spaces available
        
df = (
    df[['OccupancyDateTime', 'PaidOccupancy', 'ParkingSpaceCount'
              , 'SourceElementKey', 'BlockfaceName', 'SideOfStreet'
              , 'ParkingTimeLimitCategory', 'ParkingSpaceCount'
              , 'PaidParkingArea', 'PaidParkingSubArea', 'ParkingCategory', 'dow', 'year', 'month'
              , 'day', 'hour', 'minute']]
    .apply_rows(
        avgOccupancy
        , incols=['PaidOccupancy', 'ParkingSpaceCount']
        , outcols={'AvgOccupancy': np.float64}
        , kwargs={}
    )
)
df.head()

,OccupancyDateTime,PaidOccupancy,ParkingSpaceCount,SourceElementKey,BlockfaceName,SideOfStreet,ParkingTimeLimitCategory,PaidParkingArea,PaidParkingSubArea,ParkingCategory,dow,year,month,day,hour,minute,AvgOccupancy
0,2019-05-24 08:35:00,1.0,4.0,35438,SENECA ST BETWEEN BOYLSTON AVE AND HARVARD AVE,SE,120,First Hill,UKN,Paid Parking,4,2019,5,24,8,35,0.25
1,2019-05-24 19:57:00,3.0,3.0,57862,REPUBLICAN ST BETWEEN QUEEN ANNE AVE N AND 1ST...,S,120,Uptown,Core,Paid Parking,4,2019,5,24,19,57,1.00
2,2019-05-24 11:21:00,1.0,4.0,53542,8TH AVE BETWEEN PINE ST AND OLIVE WAY,NE,120,Commercial Core,Retail,Paid Parking,4,2019,5,24,11,21,0.25
3,2019-05-24 11:56:00,2.0,4.0,9354,BLANCHARD ST BETWEEN 1ST AVE AND 2ND AVE,SE,120,Belltown,South,Paid Parking,4,2019,5,24,11,56,0.50
4,2019-05-24 10:57:00,13.0,7.0,35889,VALLEY ST BETWEEN MINOR AVE N AND YALE AVE N,N,600,South Lake Union,North,Paid Parking,4,2019,5,24,10,57,1.00


time: 400 ms


In [21]:
def calcMean(AvgOccupancy, ParkingSpaceCount, MeanOccupancy):
    '''
        Calculate mean
    '''
    for i, (avgOccSum, avgCnt) in enumerate(zip(AvgOccupancy, ParkingSpaceCount)):
        MeanOccupancy[i] = float(avgOccSum) / avgCnt

df_agg_dt = (
    df
    .groupby(['SourceElementKey', 'dow','hour'])
    .agg({
          'ParkingSpaceCount': 'count'
        , 'AvgOccupancy': 'sum'
    })
    .reset_index()
)

df_agg_dt = df_agg_dt.apply_rows(
    calcMean
    , incols=['AvgOccupancy', 'ParkingSpaceCount']
    , outcols={'MeanOccupancy':np.float64}
    , kwargs={}
)

df_agg_dt.drop_column('AvgOccupancy')
df_agg_dt.drop_column('ParkingSpaceCount')

df_agg_dt.head().to_pandas()

,SourceElementKey,dow,hour,MeanOccupancy
0,1001,0,9,0.062500
1,1001,0,10,0.319643
2,1001,0,11,0.405357
3,1001,0,12,0.625595
4,1001,0,13,0.553571


time: 198 ms


# Find the best parking

In [22]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="todrabas_test")
location = geolocator.geocode("400 Broad St, Seattle, WA 98109") # SPACE NEEDLE

locations['LON_Ref'] = location.longitude
locations['LAT_Ref'] = location.latitude

time: 870 ms


In [23]:
from math import sin, cos, sqrt, atan2, pi

def calculateDistance(latitude, longitude, LAT_Ref, LON_Ref, Distance):
    R = 3958.8 # Earth's radius in miles
    
    for i, (lt, ln, lt_r, ln_r) in enumerate(zip(latitude, longitude, LAT_Ref, LON_Ref)):
        lt_rad = lt / 180.0 * pi
        ln_rad = ln / 180.0 * pi
        
        dlon = (ln_r - ln) / 180.0 * pi
        dlat = (lt_r - lt) / 180.0 * pi
        a = (sin(dlat/2.0))**2 + cos(lt_rad) * cos(lt_rad) * (sin(dlon/2.0))**2
        c = 2 * atan2(sqrt(a), sqrt(1-a))
        distance = R * c
        Distance[i] = distance * 5280 # in feet
        
locations = locations.apply_rows(
    calculateDistance
    , incols=['latitude', 'longitude', 'LAT_Ref', 'LON_Ref']
    , outcols={'Distance':np.float64}
    , kwargs={}
)

time: 368 ms


In [24]:
# get only meters within 1000 ft
closest = locations.query('Distance < 1000')

closest = (
    closest
    .merge(df_agg_dt, how='inner', on=['SourceElementKey'])
    .query('dow == 3 and hour == 13')
    .sort_values(by='MeanOccupancy')
)

closest_host = closest[['BlockfaceName', 'SideOfStreet'
       , 'ParkingTimeLimitCategory', 'ParkingSpaceCount', 'PaidParkingArea'
       , 'PaidParkingSubArea', 'ParkingCategory', 'Location', 'Distance'
       , 'dow', 'hour', 'MeanOccupancy', 'longitude', 'latitude']
].head().to_pandas()
closest_host

,BlockfaceName,SideOfStreet,ParkingTimeLimitCategory,ParkingSpaceCount,PaidParkingArea,PaidParkingSubArea,ParkingCategory,Location,Distance,dow,hour,MeanOccupancy,longitude,latitude
1014,5TH AVE N BETWEEN JOHN ST AND BROAD ST,W,240,3.0,Uptown Triangle,UKN,Paid Parking,POINT (-122.34771286 47.62015028),412.953708,3,13,0.433951,-122.347713,47.620150
413,2ND AVE N BETWEEN JOHN ST AND THOMAS ST,W,240,9.0,Uptown,Edge,Paid Parking,POINT (-122.35293883 47.62036489),895.541027,3,13,0.513426,-122.352939,47.620365
913,3RD AVE BETWEEN CLAY ST AND BROAD ST,NE,240,8.0,Belltown,North,Paid Parking,POINT (-122.35022717 47.61801873),937.902276,3,13,0.518519,-122.350227,47.618019
1697,3RD AVE BETWEEN CLAY ST AND BROAD ST,SW,240,8.0,Belltown,North,Paid Parking,POINT (-122.35037128 47.61790727),986.241548,3,13,0.551620,-122.350371,47.617907
879,CLAY ST BETWEEN 3RD AVE AND 4TH AVE,NW,240,9.0,Belltown,North,Paid Parking,POINT (-122.34947863 47.61806027),895.868223,3,13,0.596914,-122.349479,47.618060


time: 267 ms


# Plot the parking spots on the map

We're using gmaps python package that can be found here: https://github.com/pbugnion/gmaps. Follow the instructions contained within the README.md about how to install the package so the map shows properly in jupyter lab.

In [25]:
closest_host[['BlockfaceName', 'Distance', 'MeanOccupancy']].to_dict('records')
info_box_template = """
<dl>
<dt>Name</dt><dd>{BlockfaceName}</dd>
<dt>Distance</dt><dd>{Distance:.0f}</dd>
<dt>Occupancy (AVG)</dt><dd>{MeanOccupancy:.3f}</dd>
</dl>
"""

parking_info = [info_box_template.format(**parking) for parking in closest_host[['BlockfaceName', 'Distance', 'MeanOccupancy']].to_dict('records')]

time: 8.67 ms


In [26]:
import gmaps
from ipywidgets.embed import embed_minimal_html

####################################################
##                                                ##
## CHANGE THE API CREDS IN THE GoogleMapsAPI.cred ##
##                                                ##
####################################################
with open('config/GoogleMapsAPI.cred', 'r') as f:
    gmaps_creds = f.read()

gmaps.configure(api_key=gmaps_creds) # Your Google API key, go to https://console.developers.google.com

parking_layer = gmaps.symbol_layer(
    closest_host[['latitude', 'longitude']], fill_color="green", stroke_color="green", scale=3, info_box_content=parking_info
)

destinations_layer = gmaps.symbol_layer(
    [[location.latitude, location.longitude]]
    , info_box_content=['DESTINATION']
    , scale=5
    , fill_color="red"
    , stroke_color="red"
)

parkings = closest_host.to_dict('records')

lines_layer = gmaps.drawing_layer(features=[
    gmaps.Line(
        start= (parking['latitude'], parking['longitude'])
        , end = (location.latitude, location.longitude)
        , stroke_weight=2
        , stroke_color="red"
    )
    for parking in parkings]
)

fig = gmaps.figure(layout={'height': '500px'})
fig.add_layer(parking_layer)
fig.add_layer(destinations_layer)
fig.add_layer(lines_layer)
embed_minimal_html('maps_rendered/map_as_crow_flies.html', views=[fig])

time: 67.2 ms
